#1. 파일 읽기
> 타이타닉 파일을 읽어 들여 상위 3개의 데이터만 출력하시오

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

filepath = '/content/drive/MyDrive/7. 머신러닝 무조건 따라하기/data/titanic.csv'
df_titanic = pd.read_csv(filepath)
df_titanic.head(3)

#2. 데이터 클린징

##2.1 결측치 처리

In [3]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


> Age, Cabin, Embarked에 결측치가 발생된 것을 알 수 있다. Age는 나이로 평균을 이용하여 결측치를 해결할 것이며 나머지 두 feature는 N이라는 값을 넣을 것이다.

> 위와 같이 동작되도록 결측치를 처리해 보시오

In [5]:
round(df_titanic['Age'].mean(), 0)

30.0

> 나이는 소수점이 나올수 없음으로 평균을 구한 후 반올림 처리함

In [7]:
df_titanic['Age'].fillna(round(df_titanic['Age'].mean(), 0), inplace=True)
df_titanic['Cabin'].fillna('N', inplace=True)
df_titanic['Embarked'].fillna('N', inplace=True)

# 각 feature에 대한 합
# df_titanic.isnull().sum()
# 각 feature의 총합
df_titanic.isnull().sum().sum()

0

##2.2 이상치 처리

In [ ]:
print('성별 데이터 분포')
print(df_titanic['Sex'].value_counts())
print('객실 데이터 분포')
print(df_titanic['Cabin'].value_counts())
print('항구 데이터 분포')
print(df_titanic['Embarked'].value_counts())

> 성별 확인 결과 특이점은 없었으며 객실과 항구 정보는 이전 코드에서 변경한 데이터 이외의 특이점은 없었다.

In [ ]:
df_titanic.describe()

> 수치 데이터에서는 이상치가 발견되지 않았다.

##2.3 문자열 수치화
> 모델 생성 시 문자열이 있으면 문제가 발생할 수 있음으로 문자열을 수치 데이터로 변환한다.

In [ ]:
from sklearn import preprocessing

features = ['Cabin', 'Sex', 'Embarked']

for feature in features:
  # LabelEncode 로드
  le = preprocessing.LabelEncoder()
  # feature의 크기 설정
  le = le.fit(df_titanic[feature])
  # 데이터 변환
  df_titanic[feature] = le.transform(df_titanic[feature])

df_titanic.head()

> LabelEncoder는 문자를 수치 데이터로 변경해 준다. 내부적으로 각 데이터를 중복을 제거한 후 정렬하고 index 정보를 반환하는 기능을 한다.

> 위의 데이터에서 생존률과 관계 없는 Name과 Ticket 정보를 제거하시오

In [13]:
df_titanic.drop(['Name', 'Ticket'], axis=1, inplace=True)

#3. 모델 만들기

##3.1 데이터 분리
> 생존에 따른 데이터를 분석할 것이다. 다음 조건에 맞게 훈련용데이터와 테스트용 데이터를 추출하시오
- Label 정보는 Survived를 이용한다.
- 나머지 정보를 Data로 활용한다.
- 테스트 데이터는 30%를 사용한다.
- random_state는 62로 고정한다.

In [15]:
from sklearn.model_selection import train_test_split

# Label 정보는 Survived를 이용한다.
df_label = df_titanic['Survived']
# 나머지 정보를 Data로 활용한다.
df_data = df_titanic.drop(['Survived'], axis=1)

# 테스트 데이터는 30%를 사용한다.
# random_state는 62로 고정한다.
X_train, X_test, y_train, y_test = train_test_split(df_data, df_label, test_size=0.3, random_state=62)

##3.2 결정 트리

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# 모델 로드
dt_clf = DecisionTreeClassifier()
# 학습시키기
dt_clf.fit(X_train, y_train)
# 예측 결과 얻기
pred = dt_clf.predict(X_test)

# 정확도
print('정확도 : ', accuracy_score(y_test, pred))

정확도 :  0.7947761194029851


##3.3 랜덤 포레스트

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# 모델 로드
rf_clf = RandomForestClassifier()
# 학습시키기
rf_clf.fit(X_train, y_train)
# 예측 결과 얻기
pred = rf_clf.predict(X_test)

# 정확도
print('정확도 : ', accuracy_score(y_test, pred))

정확도 :  0.8470149253731343


##3.4 로지스틱 회귀

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# 모델 로드
lr = LogisticRegression()
# 학습시키기
lr.fit(X_train, y_train)
# 예측 결과 얻기
pred = lr.predict(X_test)

# 정확도
print('정확도 : ', accuracy_score(y_test, pred))

정확도 :  0.8171641791044776


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


> /usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

>Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

>Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

> 위와 같은 경고가 나타나면 다음과 같이 해결할 수 있다.
- lr = LogisticRegression(max_iter=1000)

> max_iter 부분에 적당한 크기의 숫자를 넣으면 경고를 제거할 수 있다.

#4. 교차검증

##4.1 cross_val_score

In [30]:
from sklearn.model_selection import cross_val_score
import numpy as np

# 교차검증을 하기 위해 호출 되었으며 인자(파라미터)는 모델, 데이터, 결과, 교차횟수 순이다.
scores = cross_val_score(dt_clf, df_data, df_label, cv=5)
print(scores)
# enumerate는 리스트의 index 정보와 데이터로 분리해서 값을 전달
for iter_cnt, accuracy in enumerate(scores):
  # format은 {}(괄호)안의 정보에 들어갈 값을 지정한다. {}(괄호)안의 숫자는 위치를 의미한다.
  # .4f는 소수점 이하 4자리까지 표현하며 f는 실수를 의미한다.
  print('교차 검증 {0} 정확도 : {1:.4f}'.format(iter_cnt, accuracy))

print('평균 정확도 : {0: .2f}'.format(np.mean(scores)))

[0.5698324  0.79213483 0.80337079 0.74719101 0.83707865]
교차 검증 0 정확도 : 0.5698
교차 검증 1 정확도 : 0.7921
교차 검증 2 정확도 : 0.8034
교차 검증 3 정확도 : 0.7472
교차 검증 4 정확도 : 0.8371
평균 정확도 :  0.75


In [34]:
def cross_score(clf):
  scores = cross_val_score(clf, df_data, df_label, cv=5)
  
  for iter_cnt, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도 : {1:.4f}'.format(iter_cnt, accuracy))

  print('평균 정확도 : {0: .2f}'.format(np.mean(scores)))

cross_score(dt_clf)
cross_score(rf_clf)
cross_score(lr)

교차 검증 0 정확도 : 0.5754
교차 검증 1 정확도 : 0.7753
교차 검증 2 정확도 : 0.8090
교차 검증 3 정확도 : 0.7640
교차 검증 4 정확도 : 0.8315
평균 정확도 :  0.75
교차 검증 0 정확도 : 0.7430
교차 검증 1 정확도 : 0.8090
교차 검증 2 정확도 : 0.8483
교차 검증 3 정확도 : 0.7978
교차 검증 4 정확도 : 0.8427
평균 정확도 :  0.81
교차 검증 0 정확도 : 0.7263
교차 검증 1 정확도 : 0.8090
교차 검증 2 정확도 : 0.7584
교차 검증 3 정확도 : 0.7528
교차 검증 4 정확도 : 0.8146
평균 정확도 :  0.77


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


> 교차 검증을 통해 랜덤 포레스트 모델의 성능이 더 좋다는 것을 확인했다.

#5. 튜닝

##5.1 트리 관련

In [32]:
from sklearn.model_selection import GridSearchCV

# 의사결정트리, 랜덤 포레스트의 파라미터
parameters = {
    'max_depth':[2,3,5,10],
    'min_samples_split':[2,3,5],
    'min_samples_leaf':[1,5,8]
}

grid_clf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_clf.fit(X_train, y_train)

print('최적의 파라미터 : ', grid_clf.best_params_)
print('최고의 정확도 : ', grid_clf.best_score_)

최적의 파라미터 :  {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
최고의 정확도 :  0.7897419354838711


In [33]:
grid_clf = GridSearchCV(rf_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_clf.fit(X_train, y_train)

print('최적의 파라미터 : ', grid_clf.best_params_)
print('최고의 정확도 : ', grid_clf.best_score_)

최적의 파라미터 :  {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 3}
최고의 정확도 :  0.8122064516129033


##5.2 로지스틱 회귀

In [36]:
parameters={
    'C':[2,3,5,10],
    'penalty':['l2'],
    'max_iter':[1000]
}

grid_clf = GridSearchCV(lr, param_grid=parameters, scoring='accuracy', cv=5)
grid_clf.fit(X_train, y_train)

print('최적의 파라미터 : ', grid_clf.best_params_)
print('최고의 정확도 : ', grid_clf.best_score_)

최적의 파라미터 :  {'C': 2, 'max_iter': 1000, 'penalty': 'l2'}
최고의 정확도 :  0.796167741935484
